## 経験財データセット

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# 認証のためのコード
from google.colab import auth
auth.authenticate_user()


Mounted at /content/drive


In [ ]:
!pip install gspread

In [ ]:

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

def readSheet(id, sheet_name):
  workbook = gc.open_by_key(id)
  worksheet = workbook.worksheet(sheet_name)
  return worksheet.get_all_values()


In [ ]:
ss_id = "スプレッドシートのIDを入力"#経験財データセット
ss_aug_id = "スプレッドシートのIDを入力"
#ss_single_label_id = "17_IKpIwaXOoDWsh8aPQeN7MqEBan7BYPMqrU_wU8LJ8"

In [ ]:
workbook = gc.open_by_key(ss_id)
worksheet = workbook.worksheet("シート2")
print(worksheet.get_all_values())

[['タイトル', 'レビュー', 'カテゴリー', '', '商品名', '', 'URL', ''], ['5つ星のうち5.0 初心者向けだが、ゼルダマニアでも楽しめる斬新さ', '初心者向けだが、ゼルダマニアでも楽しめる斬新さ\n\n姫が主人公と言うことで即買いしましたが\n今までのゼルダシリーズとは違って、新鮮！\n\nリンクは基本剣弓盾で冒険するにあたって\nタイトル通り、様々なオブジェクトや魔物をコピー（借りて）して冒険。\n\nコピーした魔物を召喚して戦うから\n今までにない感じで楽しい。\nそして姫が可愛い。\n\n強いて言うなら、かなり初心者向けでもある。\n✿謎解きが割と全部簡単\n✿フィールドから落ちてもHP減らない\n✿ベッド召喚して、いつどこでもお手軽HP回復\n\nシリーズによるけど、コンパスない分\nダンジョンマップ手に入れたら宝箱も自動的に表示されるから取り損ねがない。\n\nブレティアのオープンワールドの良さを活かして\n正規ルート以外も進み放題！\nこれが割とビックリしたw\n\nただ他の人も言う通り\nリンクモード（剣弓盾攻撃）は正直いらないかも。\nあったら楽なのは分かるけど姫だけでも十分戦える！\n\nあと、やり込みたい人には向かない…かも？\nストーリーは24時間あればクリア出来ます。\nまぁ自分含めサブクエとか諸々全部やりたい派には関係ないけど！\n11人のお客様がこれが役に立ったと考えています', '1,2,5', '', 'ゼルダの伝説 知恵のかりもの - Switch', '', 'https://www.amazon.co.jp/%E4%BB%BB%E5%A4%A9%E5%A0%82-HAC-P-BDGEA-%E3%82%BC%E3%83%AB%E3%83%80%E3%81%AE%E4%BC%9D%E8%AA%AC-%E7%9F%A5%E6%81%B5%E3%81%AE%E3%81%8B%E3%82%8A%E3%82%82%E3%81%AE-Switch/dp/B0D7H1VFKB/ref=sr_1_5?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&crid=M9RROX0ENX8N&dib=eyJ2IjoiMSJ9.Z7ErKDBePEPTuABpfk27oqq

In [ ]:
"""
・5月10日 API get_all_record　がバグっているらしくて泣く泣くget_all_values()を使用
・それに伴いformat_list関数変更
"""
from os import replace
import re
import csv
def format_list(review : list) -> list:
  review_list = []
  patturn = '\d+\.\d\s' #タイトル内　星数取得のためのパターン
  patturn1 = '\d+人のお客様がこれが役に立ったと考えています'
  patturn2 = '人のお客様がこれが役に立ったと考えています'
  del review[0] #['タイトル', 'レビュー', 'カテゴリー', '', '商品名', '', 'URL']　こいつ削除
  for i in range(len(review)):
      dict = {}
      star, useful_num = 0, 0
      useful_text = ''
      dict["タイトル"] = review[i][0]
      dict['タイトル'] = re.sub('5つ星のうち', '', dict['タイトル'])
      star = re.match(patturn, dict['タイトル'])
      dict['星数'] = float(star.group())
      dict['タイトル'] = re.sub(patturn, '', dict['タイトル'])
      #dict['レビュー'] = f'{dict["タイトル"]}\n{review[i][1]}'
      dict['レビュー'] = f'{review[i][1]}'
      dict['カテゴリー'] = review[i][2]
      dict['参考になった数'] = review[i][3]
      useful_text = re.search(patturn1, dict['レビュー'])
      if useful_text != None:
        useful_text = useful_text.group()
        dict['レビュー'] = re.sub(useful_text, '', dict['レビュー'])
        useful_num = re.sub(patturn2, '', useful_text)
      else:
        useful_num = 0

      dict['参考になった数'] = int(useful_num)
      review_list.append(dict)
  return review_list

In [ ]:
sheet1_list = format_list(readSheet(ss_id,"シート1"))
sheet2_list = format_list(readSheet(ss_id,"シート2"))
sheet3_list = format_list(readSheet(ss_id,"シート3"))
sheet4_list = format_list(readSheet(ss_id,"シート4"))
sheet5_list = format_list(readSheet(ss_id,"シート5"))
sheet6_list = format_list(readSheet(ss_id,"シート6"))
#sheet8_list = format_list(readSheet(ss_id,"シート7"))
#sheet9_list = format_list(readSheet(ss_id,"シート8"))
all_list = sheet1_list + sheet2_list + sheet3_list + sheet4_list + sheet5_list + sheet6_list #+ sheet7_list + sheet8_list + sheet9_list
#all_list = format_list(all_list)


In [ ]:
#オーグメンテーション用

aug_list = readSheet(ss_aug_id,"シート1")
aug_list = format_list(aug_list)


In [ ]:
#シングルラベルレビュー用
"""
single_label_list = readSheet(ss_single_label_id,"シート1")
single_label_list = format_list(single_label_list)
"""

'\nsingle_label_list = readSheet(ss_single_label_id,"シート1")\nsingle_label_list = format_list(single_label_list)\n'

In [ ]:
len(all_list)

564

In [ ]:
"""
#pattrunマッチ確認用
patturn1 = '\d人のお客様がこれが役に立ったと考えています'
text = '寝室に置いておくのに丁度良いサイズ\n2人のお客様がこれが役に立ったと考えています'
usefull_text = re.search(patturn1, text)
print(usefull_text)
"""

"\n#pattrunマッチ確認用\npatturn1 = '\\d人のお客様がこれが役に立ったと考えています'\ntext = '寝室に置いておくのに丁度良いサイズ\n2人のお客様がこれが役に立ったと考えています'\nusefull_text = re.search(patturn1, text)\nprint(usefull_text)\n"

In [ ]:
# CSVファイルの保管先
csv_path = "/content/drive/MyDrive/"
# CSVファイル名
csv_name = 'experience_review_ver20241105.csv'
field_name = ["タイトル","レビュー","カテゴリー", "星数", "参考になった数"]
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(all_list)

In [ ]:

#オーグメンテーション用
# CSVファイルの保管先
csv_path = "/content/drive/MyDrive/"
# CSVファイル名
csv_name = 'aug_experience_review_ver20241105.csv'
field_name = ["タイトル","レビュー","カテゴリー", "星数", "参考になった数"]
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(aug_list)


In [ ]:
"""
#シングルラベルレビュー用
# CSVファイルの保管先
csv_path = "/content/drive/MyDrive/"
# CSVファイル名
csv_name = 'single_label_review.csv'
field_name = ["タイトル","レビュー","カテゴリー", "星数", "参考になった数"]
# リストをCSVファイルに出力（'w'の場合は、新規作成）
with open(csv_path+csv_name, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames = field_name)
    writer.writeheader()
    writer.writerows(single_label_list)

"""

'\n#シングルラベルレビュー用\n# CSVファイルの保管先\ncsv_path = "/content/drive/MyDrive/"\n# CSVファイル名\ncsv_name = \'single_label_review.csv\'\nfield_name = ["タイトル","レビュー","カテゴリー", "星数", "参考になった数"]\n# リストをCSVファイルに出力（\'w\'の場合は、新規作成）\nwith open(csv_path+csv_name, \'w\', newline=\'\') as f:\n    writer = csv.DictWriter(f, fieldnames = field_name)\n    writer.writeheader()\n    writer.writerows(single_label_list)\n\n'